In [2]:
import minsearch
import json
import os
from tqdm.auto  import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration

from elasticsearch import Elasticsearch



In [3]:


es_client = Elasticsearch('http://localhost:9200')

es_client.info()



ObjectApiResponse({'name': 'e359b3f82038', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'LuNImNV6RB2lHk25Z9NgRA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small",cache_dir='models')
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto",cache_dir='models')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
q = "how do I run kafka?"


index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": q,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}


prompt_template = """
You're a course teaching assistant. Answer the question based on the context from the FAQ database.
Use only the facts from the CONTEXT when answering the Question.

QUESTION: {question}

CONTEXT: {context}

""".strip()





In [6]:
def minsearch_result(query):

    with open('../../data/documents.json','rt') as f_in:
        docs_raw = json.load(f_in)

    documents = []


    for course_dict in docs_raw:
        for doc in course_dict['documents']:
            doc['course'] = course_dict['course']
            documents.append(doc)
            

    index = minsearch.Index(
        text_fields=["question","text","section"],
        keyword_fields=['course']
        )

    index.fit(documents)



    boost = { 'question':3.0,'section':0.5}

    search_results = index.search(
            query=query,
            filter_dict={'course':'data-engineering-zoomcamp'},
            boost_dict=boost,
            num_results=5
        )
    
    return search_results



In [7]:
def elastic_search_result(query):

    index_name = "course-questions"

    try:
       es_client.indices.create(index=index_name,body=index_settings)
    except:
       pass
    



    with open('../../data/documents.json','rt') as f_in:
        docs_raw = json.load(f_in)

    documents = []


    for course_dict in docs_raw:
        for doc in course_dict['documents']:
            doc['course'] = course_dict['course']
            documents.append(doc)

    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)

    response = es_client.search(index=index_name,body=query)

    result_docs = []


    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [8]:
def create_context(search_results):
    context = ""


    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return context

In [9]:
def  create_prompt(context,question,prompt_template):


    prompt = prompt_template.format(question=question,context=context).strip()

    return prompt

In [10]:
def get_query_result(prompt,generate_params=None):
    
    
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [11]:
def rag(query,prompt_template,search_engine=None):
    if search_engine == "minsearch":
       search_results = minsearch_result(query)
    else:
       search_results = elastic_search_result(query)

    context = create_context(search_results)
    prompt = create_prompt(context=context,question=query,prompt_template=prompt_template)
    print(prompt)
    
    return get_query_result(prompt)

In [12]:
rag(q,prompt_template,"minsearch")

Token indices sequence length is longer than the specified maximum sequence length for this model (679 > 512). Running this sequence through the model will result in indexing errors


You're a course teaching assistant. Answer the question based on the context from the FAQ database.
Use only the facts from the CONTEXT when answering the Question.

QUESTION: how do I run kafka?

CONTEXT: section: Module 6: streaming with kafka
question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

section: Module 6: streaming with kafka
question: Module “kafka” not found when trying to run producer.py
answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual env):
source env/bin/activate
To deactivate it:
deactivate
This works on MacOS, Linux and Windows - b

/Users/adebimpe/Desktop/llm_zoomcamp/llm_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [15]:
rag(search_query,prompt_template)

100%|██████████| 948/948 [00:01<00:00, 658.03it/s]


You're a course teaching assistant. Answer the question based on the context from the FAQ database.
Use only the facts from the CONTEXT when answering the Question.

QUESTION: {'size': 5, 'query': {'bool': {'must': {'multi_match': {'query': 'how do I run kafka?', 'fields': ['question^3', 'text', 'section'], 'type': 'best_fields'}}, 'filter': {'term': {'course': 'data-engineering-zoomcamp'}}}}}

CONTEXT: section: Workshop 1 - dlthub
question: How do I install the necessary dependencies to run the code?
answer: Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).

section: Workshop 1 - dlthub
question: How do I install the necessary dependencies to run the code?
answer: Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. 

'In the context of the Data Engineering Zoomcamp, here is how you can run Kafka:\n\nFor Java-based Kafka, to run producer/consumer/kstreams/etc in the terminal, navigate to your project directory and execute the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your jar file.'